# Read and write CPACS using TiXI

[TiXI](https://github.com/DLR-SC/tixi):
 - Binary Downloads:  https://github.com/DLR-SC/tixi/wiki/Downloads
 - API Documentation: http://tixi.sourceforge.net/Doc/index.html
 - Issue Tracker:     https://github.com/DLR-SC/tixi/issues
 - Wiki:              https://github.com/DLR-SC/tixi/wiki

In [ ]:
from tixi3 import tixi3wrapper
tixi_h = tixi3wrapper.Tixi3()

In [ ]:
help(tixi_h)

Open a CPACS file with tixi:

In [ ]:
tixi_h.open('../D150_v30.xml')

Get airport information:

In [ ]:
#airport_xpath = '/cpacs/airports/airport[@uID="FRA"]'
airport_xpath = tixi_h.uIDGetXPath('FRA')

airport_name = tixi_h.getTextElement('/'.join((airport_xpath, 'name')))
positionNorth = tixi_h.getDoubleElement('/'.join((airport_xpath, 'positionNorth')))
positionEast = tixi_h.getDoubleElement('/'.join((airport_xpath, 'positionEast')))

print('Name: %s\nPosition North: %.3f\nPosition East: %.3f' % (airport_name, positionNorth, positionEast))

Some information would be very long if stored as xml-sequence. Thus, we introduced vectors:

In [ ]:
root_xpath = '/'.join((tixi_h.uIDGetXPath('FRAtoEWRID'),'flightPath'))

vector_size = tixi_h.getVectorSize('/'.join((root_xpath,'waypoints')))

waypoints = tixi_h.getTextElement('/'.join((root_xpath,'waypoints'))).split(';')
latitude = tixi_h.getFloatVector('/'.join((root_xpath,'latitude')), vector_size)
longitude = tixi_h.getFloatVector('/'.join((root_xpath,'longitude')), vector_size)

for i, wp in enumerate(waypoints):
    print('%10s %.3f %.3f'%(wp, latitude[i], longitude[i]))

Let's add information about Braunschweig airport:
    - add uID="BWE" to the new <airport> element
    - add text element "name" with "Braunschweig, Germany"
    - add Positions: 52.3199° north / 10.556° east
    - add elevation of 276ft

In [ ]:
apts_xpath = '/cpacs/airports'
idx = tixi_h.getNamedChildrenCount(apts_xpath,'airport') + 1

tixi_h.createElementAtIndex(apts_xpath,'airport',idx)

apt_xpath = '/'.join((apts_xpath,'airport[%i]'%idx))

tixi_h.addTextAttribute(apt_xpath,'uID','BWE')
tixi_h.addTextElement(apt_xpath, 'name', 'Braunschweig, Germany')
tixi_h.addDoubleElement(apt_xpath, 'positionNorth', 52.319, '%.3f')
tixi_h.addDoubleElement(apt_xpath, 'positionEast', 10.556, '%.3f')
tixi_h.addIntegerElement(apt_xpath, 'elevation', 84, '%i')

Save the modified CPACS instance as a new file:

In [ ]:
tixi_h.save('D150_v30_modified.xml')
tixi_h.close()